# Get encoded atoms

In [1]:
import pandas as pd
import os

import sys
sys.path.append('../../../scripts/SOTA/jtree/')
# sys.path.append('../../../../icml18-jtnn/jtnn')

In [2]:
from tqdm import tqdm

In [16]:
VOCAB_PATH = '../../../data/raw/baselines/jtree/vocab.txt'
MODEL_PATH = '../../../data/raw/baselines/jtree'
DATASET_PATH = '../../../data/3_final_data/split_data'

RAW_PATH = '../../../data/raw/baselines/jtree'

SMILES_COLUMN = 'smiles'

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np

import math, random, sys
from optparse import OptionParser
from collections import deque

import rdkit
import rdkit.Chem as Chem
from mol_tree import MolTree, Vocab
import numpy as np
from jtnn_enc import JTNNEncoder
from mpn_more_atom_feats import MPN,  mol2graph


In [5]:
vocab = [x.strip("\r\n ") for x in open(VOCAB_PATH)] 
vocab = Vocab(vocab)

In [6]:
hidden_size = int(450)
latent_size = int(56)
depth = int(3)
stereo = True if int(1) == 1 else False

In [8]:
from jtnn_vae import JTNNVAE
model = JTNNVAE(vocab, hidden_size, latent_size, depth, stereo=stereo)
model.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'MPNVAE-h450-L56-d3-beta0.001/model.iter-4')))
model = model.cuda()

/home/mol/anaconda3/envs/jtree/lib/python2.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [11]:
def create_latent_representation_dataset(df, SMILES_COLUMN, VALUE_COLUMN):
    import numpy as np    
    vectors = []
    broken_smiles = {}
    for smiles in tqdm(df[SMILES_COLUMN].values):
        try:
            latent_representation = model.encode_latent_mean([smiles])
        except (KeyError, RuntimeError), e:
            broken_smiles[smiles]=e
            continue
        vectors.append(latent_representation.cpu().detach().numpy())
    X = np.array(vectors)
    y = df[~df[SMILES_COLUMN].isin(list(broken_smiles.keys()))][VALUE_COLUMN].values
    assert len(X)==len(y)
    return X, y, broken_smiles

In [17]:
datasets_prefix = ['logp_wo_logp_json_wo_averaging', 'logd_Lip_wo_averaging']
VALUE_COLS = ['logP', 'logD']

In [18]:
X_trains = []
y_trains = []
trains_errs = []
X_vals = []
y_vals = []
vals_errs = []
X_tests = []
y_tests = []
tests_errs = []

for VALUE_COLUMN,dataset_prefix in zip(VALUE_COLS, datasets_prefix):
    dataset_train = pd.read_csv(os.path.join(DATASET_PATH, dataset_prefix+'_train.csv'), index_col=0)
    dataset_val = pd.read_csv(os.path.join(DATASET_PATH, dataset_prefix+'_validation.csv'), index_col=0)
    dataset_test = pd.read_csv(os.path.join(DATASET_PATH, dataset_prefix+'_test.csv'), index_col=0)
    X_train, y_train, train_errs = create_latent_representation_dataset(dataset_train, SMILES_COLUMN, VALUE_COLUMN)
    X_trains.append(X_train)
    y_trains.append(y_train)
    trains_errs+=train_errs

    X_val, y_val, val_errs = create_latent_representation_dataset(dataset_val, SMILES_COLUMN, VALUE_COLUMN)
    X_vals.append(X_val)
    y_vals.append(y_val)
    vals_errs+=val_errs
    
    X_test, y_test, test_errs = create_latent_representation_dataset(dataset_test, SMILES_COLUMN, VALUE_COLUMN)
    X_tests.append(X_test)
    y_tests.append(y_test)
    tests_errs+=test_errs

100%|██████████| 625/625 [00:43<00:00, 14.46it/s]


In [20]:
with open(os.path.join(RAW_PATH,'X_train.npy'), 'wb') as f:
    np.save(f,np.concatenate(X_trains, axis = 0))
with open(os.path.join(RAW_PATH,'y_train.npy'), 'wb') as f:
    np.save(f,np.concatenate(y_trains, axis = 0))  
    
with open(os.path.join(RAW_PATH,'X_val.npy'), 'wb') as f:
    np.save(f,np.concatenate(X_vals, axis = 0))
with open(os.path.join(RAW_PATH,'y_val.npy'), 'wb') as f:
    np.save(f,np.concatenate(y_vals, axis = 0))

with open(os.path.join(RAW_PATH,'X_test.npy'), 'wb') as f:
    np.save(f,np.concatenate(X_tests, axis = 0))
with open(os.path.join(RAW_PATH,'y_test.npy'), 'wb') as f:
    np.save(f,np.concatenate(y_tests, axis = 0))   

In [23]:
with open(os.path.join(RAW_PATH,'train_errs.txt'), 'w') as f:
    f.write('\n'.join(trains_errs))
with open(os.path.join(RAW_PATH,'val_errs.txt'), 'w') as f:
    f.write('\n'.join(vals_errs))
with open(os.path.join(RAW_PATH,'test_errs.json'), 'w') as f:
    f.write('\n'.join(tests_errs))

# Train model based on pretrained vectors 

## Pytorch NN

In [13]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import roc_auc_score, r2_score, mean_squared_error

import os

from tqdm import tqdm

from sklearn.preprocessing import StandardScaler

RAW_PATH = '../../../data/raw/baselines/jtree'

scaler = StandardScaler()

In [18]:
fname = os.path.join(RAW_PATH, 'logs', 'exp_0')
os.makedirs(fname)
writer = SummaryWriter(fname)

In [19]:
from tensorboardX import SummaryWriter

In [60]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter:',self.counter,' out of ',self.patience)
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print('Validation loss decreased (', round(self.val_loss_min, 6), '-->', round(val_loss, 6),').  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [62]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [70]:
def train(model, loader, optimizer, device):
    model.train()

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)

        #Whether y is non-null or not.
        is_valid = y**2 > 0
        #Loss matrix
        loss = criterion(y_pred, y)
        #loss matrix after removing null target
        # loss_mat = torch.where(is_valid, loss_mat, torch.zeros(loss_mat.shape).to(loss_mat.device).to(loss_mat.dtype))
        #
        optimizer.zero_grad()
        # loss = torch.sum(loss_mat)/torch.sum(is_valid)
        loss.backward()

        optimizer.step()


def eval(model, loader, scaler, device,  train = False):
    model.eval()
    y_true = []
    y_scores = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        

        with torch.no_grad():
            pred = model(x)

        y_true.append(y.squeeze())
        if train:
            y_scores.append(pred.squeeze())
        else:
            y_scores.append(torch.Tensor(scaler.inverse_transform(pred.squeeze().cpu())))

    y_true = torch.cat(y_true, dim = 0).cpu().numpy()
    y_scores = torch.cat(y_scores, dim = 0).cpu().numpy()

    r2 = r2_score(y_true, y_scores)
    rmse = mean_squared_error(y_true, y_scores)**0.5

    return r2, rmse #y_true.shape[1]

In [71]:
D_in, H, D_out = 56, 100, 1

device = torch.device("cuda:0")

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)
model = model.to(device)


# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [72]:
with open(os.path.join(RAW_PATH,'X_train.npy'), 'rb') as f:
    X_train = np.load(f)
with open(os.path.join(RAW_PATH,'y_train.npy'), 'rb') as f:
    y_train = np.load(f)
    
X_train = torch.Tensor(X_train) # transform to torch tensor
# y_train = torch.Tensor(y_train)

y_train_transformed = torch.Tensor(scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1))

train_loader = DataLoader(TensorDataset(X_train,y_train_transformed), batch_size = 32) # create your dataloader
    
with open(os.path.join(RAW_PATH,'X_val.npy'), 'rb') as f:
    X_val = np.load(f)
with open(os.path.join(RAW_PATH,'y_val.npy'), 'rb') as f:
    y_val = np.load(f)
    
X_val = torch.Tensor(X_val) # transform to torch tensor
y_val = torch.Tensor(y_val)

val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size = 32) # create your dataloader

with open(os.path.join(RAW_PATH,'X_test.npy'), 'rb') as f:
    X_test = np.load(f)
with open(os.path.join(RAW_PATH,'y_test.npy'), 'rb') as f:
    y_test = np.load(f)
    
X_test = torch.Tensor(X_test) # transform to torch tensor
y_test = torch.Tensor(y_test)

test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size = 32) # create your dataloader

In [73]:
early_stopping = EarlyStopping(patience=50, verbose=True, path=os.path.join(fname, 'chkpnt' + '.pth'))

In [74]:
for epoch in range(1, 100):
        print("====epoch " + str(epoch))
        
        train(model, train_loader, optimizer, device)

        print("====Evaluation")

        train_r2, train_rmse = eval(model, train_loader, scaler,device,  train = True)
        
        val_r2, val_rmse = eval(model, val_loader, scaler, device)
        test_r2, test_rmse = eval( model, test_loader, scaler, device)

        print("train r2: %f\ntrain rmse: %f\n val r2: %f\n val rmse: %f\ntest r2: %f\ntest rmse: %f"\
              %(train_r2, train_rmse, val_r2, val_rmse, test_r2, test_rmse))

        # val_acc_list.append(val_acc)
        # test_acc_list.append(test_acc)
        # train_acc_list.append(train_acc)

#         if not args.filename == "":
        writer.add_scalar('data/train r2', train_r2, epoch)
        writer.add_scalar('data/train rmse', train_rmse, epoch)

        writer.add_scalar('data/val r2', val_r2, epoch)
        writer.add_scalar('data/val rmse', val_rmse, epoch)
        writer.add_scalar('data/test r2', test_r2, epoch)
        writer.add_scalar('data/test rmse', test_rmse, epoch)

        early_stopping(val_rmse, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

        print("")


Iteration:  13%|█▎        | 38/295 [00:00<00:00, 361.28it/s]

====epoch 1


Iteration:  33%|███▎      | 98/295 [00:00<00:00, 973.92it/s]

====Evaluation


Iteration:  12%|█▏        | 34/295 [00:00<00:00, 336.07it/s]

...

====Evaluation


Iteration:  12%|█▏        | 34/295 [00:00<00:00, 337.68it/s]

train r2: 0.055955
train rmse: 0.971620
 val r2: 0.041619
 val rmse: 1.785370
test r2: 0.056788
test rmse: 1.773341
('EarlyStopping counter:', 20, ' out of ', 50)

====epoch 24


Iteration:  37%|███▋      | 109/295 [00:00<00:00, 347.12it/s]


KeyboardInterrupt: 

## XGBoost

In [11]:
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [14]:
with open(os.path.join(RAW_PATH,'X_train.npy'), 'rb') as f:
    X_train = np.load(f).squeeze()
with open(os.path.join(RAW_PATH,'y_train.npy'), 'rb') as f:
    y_train = np.load(f)
    
y_train_transformed = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)
    
with open(os.path.join(RAW_PATH,'X_val.npy'), 'rb') as f:
    X_val = np.load(f).squeeze()
with open(os.path.join(RAW_PATH,'y_val.npy'), 'rb') as f:
    y_val = np.load(f)

with open(os.path.join(RAW_PATH,'X_test.npy'), 'rb') as f:
    X_test = np.load(f).squeeze()
with open(os.path.join(RAW_PATH,'y_test.npy'), 'rb') as f:
    y_test = np.load(f) 

### Train with whole vector

In [15]:
model = XGBRegressor(n_estimators = 100, max_depth = 6, random_state = 42)

In [16]:
y_train_transformed = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)

In [17]:
model.fit(X_train, y_train_transformed)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [18]:
test_preds = scaler.inverse_transform(model.predict(X_test))
val_preds = scaler.inverse_transform(model.predict(X_val))

In [19]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds)**0.5)
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds)**0.5)
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

('Valid RMSE =', 0.905357986166681)
Valid R2-score is 0.753553741911
('Test RMSE =', 0.9247779104382)
Test R2-score is 0.743492953339


In [20]:
feature_importance = model.feature_importances_
features = list(range(X_train.shape[1]))
df = pd.DataFrame(columns = ['feature importance', 'feature name'])
df['feature importance'] = feature_importance
df['feature name'] = [str(i)+ '_tree_feats' for i in range(X_train.shape[1]//2)] +\
                    [str(i)+ '_graph_feats' for i in range(X_train.shape[1]//2, X_train.shape[1])]
df_20_most_important = df.sort_values(by='feature importance', ascending = False)[:20]

In [7]:
import pandas as pd
import os

In [8]:
df_20_most_important = pd.read_csv(os.path.join(RAW_PATH,'experiments_result.csv'), index_col = 0)

In [13]:
for i in range(len(df_20_most_important)):
    print '|', df_20_most_important.iloc[i]['feature name'], '|', df_20_most_important.iloc[i]['feature importance'], '|'

| 2_tree_feats | 0.13753037 |
| 17_tree_feats | 0.11225141 |
| 22_tree_feats | 0.061198685 |
| 27_tree_feats | 0.040304124 |
| 21_tree_feats | 0.03701255 |
| 10_tree_feats | 0.029927588999999997 |
| 16_tree_feats | 0.026088234 |
| 32_graph_feats | 0.02460154 |
| 3_tree_feats | 0.024430856 |
| 24_tree_feats | 0.02399634 |
| 42_graph_feats | 0.021596342 |
| 26_tree_feats | 0.021326277 |
| 1_tree_feats | 0.021274897999999997 |
| 19_tree_feats | 0.018548844 |
| 15_tree_feats | 0.017468281000000002 |
| 29_graph_feats | 0.01733831 |
| 36_graph_feats | 0.015398215 |
| 8_tree_feats | 0.014655213 |
| 12_tree_feats | 0.014508405 |
| 13_tree_feats | 0.013611487 |


### Train with tree features

In [133]:
model = XGBRegressor(n_estimators = 100, max_depth = 6, random_state = 42)

In [ ]:
y_train_transformed = scaler.fit_transform(y_train.reshape(-1, 1)).reshape(-1)

In [135]:
model.fit(X_train[:, :X_train.shape[1]//2], y_train_transformed)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [136]:
test_preds = scaler.inverse_transform(model.predict(X_test[:, :X_train.shape[1]//2]))
val_preds = scaler.inverse_transform(model.predict(X_val[:, :X_train.shape[1]//2]))

In [137]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds)**0.5)
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds)**0.5)
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

('Valid RMSE =', 0.9402256664953701)
Valid R2-score is 0.734205640941
('Test RMSE =', 0.9550508773345758)
Test R2-score is 0.726424361726


In [121]:
feature_importance = model.feature_importances_
df = pd.DataFrame(columns = ['feature importance', 'feature name'])
df['feature importance'] = feature_importance
df['feature name'] = [str(i)+ '_tree_feats' for i in range(X_train.shape[1]//2)]
df_20_most_important = df.sort_values(by='feature importance', ascending = False)[:20]

In [122]:
df_20_most_important

,feature importance,feature name
2,0.176946,2_tree_feats
17,0.152939,17_tree_feats
22,0.080504,22_tree_feats
27,0.051779,27_tree_feats
21,0.047035,21_tree_feats
3,0.037569,3_tree_feats
10,0.037453,10_tree_feats
16,0.037348,16_tree_feats
24,0.031914,24_tree_feats
26,0.030068,26_tree_feats


### Train with graph features

In [123]:
model = XGBRegressor(n_estimators = 100, max_depth = 6, random_state = 42)

In [124]:
model.fit(X_train[:, X_train.shape[1]//2:], y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [125]:
test_preds = model.predict(X_test[:, X_train.shape[1]//2:])
val_preds = model.predict(X_val[:, X_train.shape[1]//2:])

In [126]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds)**0.5)
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds)**0.5)
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

('Valid RMSE =', 1.3417327115036215)
Valid R2-score is 0.458730481237
('Test RMSE =', 1.3324128192277744)
Test R2-score is 0.467521581709


In [127]:
feature_importance = model.feature_importances_
df = pd.DataFrame(columns = ['feature importance', 'feature name'])
df['feature importance'] = feature_importance
df['feature name'] = [str(i)+ '_graph_feats' for i in range(X_train.shape[1]//2, X_train.shape[1])]
df_20_most_important = df.sort_values(by='feature importance', ascending = False)[:20]

In [128]:
df_20_most_important

,feature importance,feature name
27,0.055859,55_graph_feats
22,0.051214,50_graph_feats
4,0.050143,32_graph_feats
8,0.049575,36_graph_feats
11,0.048136,39_graph_feats
14,0.046766,42_graph_feats
9,0.044520,37_graph_feats
18,0.042289,46_graph_feats
13,0.039642,41_graph_feats
21,0.038473,49_graph_feats


### Grid search

In [152]:
from sklearn.model_selection import GridSearchCV
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 10, 20, 50],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100, 200, 500, 1000]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(np.concatenate((X_train, X_val), axis=0),
         np.concatenate((y_train, y_val), axis=0))

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed: 14.8min
[Parallel(n_jobs=5)]: Done  96 out of  96 | elapsed: 48.1min finished


0.7722961415729136
{'colsample_bytree': 0.7, 'silent': 1, 'learning_rate': 0.05, 'nthread': 4, 'min_child_weight': 4, 'n_estimators': 1000, 'subsample': 0.7, 'max_depth': 5}


In [156]:
model = XGBRegressor(colsample_bytree=0.7, silent = 1, learning_rate = 0.05, nthread = 4, min_child_weight = 4, n_estimators = 1000, subsample = 0.7, max_depth = 5)

In [157]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, importance_type='gain',
       learning_rate=0.05, max_delta_step=0, max_depth=5,
       min_child_weight=4, missing=None, n_estimators=1000, n_jobs=1,
       nthread=4, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.7)

In [158]:
test_preds = model.predict(X_test)
val_preds = model.predict(X_val)

In [159]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds)**0.5)
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds)**0.5)
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

('Valid RMSE =', 0.8063512258633068)
Valid R2-score is 0.804507520131
('Test RMSE =', 0.8211428563420121)
Test R2-score is 0.797762427624


In [1]:
test_RMSE = [0.5342044580494232, 0.5405131737041414, 0.5144339590477948]
test_R2 = [0.9212916061654113, 0.9134389008189245, 0.9156177478399877]

val_RMSE = [0.5330971133864346, 0.5393970943801238, 0.5320911200134589]
val_R2 = [0.9167385707444774, 0.9193261655969616, 0.9188716678036257]

train_RMSE = [0.18383073729520522, 0.18567786361473315, 0.17486524864784958]
train_R2 = [0.9662330071097933, 0.9655380319633392, 0.969380081805695]

import numpy as np

In [5]:
print("Train RMSE =", np.mean(train_RMSE),'+-',np.std(train_RMSE))
print("Train R2-score is {0}+-{1}".format(np.mean(train_R2),np.std(train_R2)))

print("Valid RMSE =", np.mean(val_RMSE),'+-',np.std(val_RMSE))
print("Valid R2-score is {0}+-{1}".format(np.mean(val_R2),np.std(val_R2)))

print("Test RMSE =", np.mean(test_RMSE),'+-',np.std(test_RMSE))
print("Test R2-score is {0}+-{1}".format(np.mean(test_R2),np.std(test_R2)))

Train RMSE = 0.18145794985259597 +- 0.004722340577254158
Train R2-score is 0.9670503736262758+-0.0016716065654506672
Valid RMSE = 0.5348617759266725 +- 0.0032331450913189994
Valid R2-score is 0.9183121347150216+-0.0011280424749930953
Test RMSE = 0.5297171969337865 +- 0.011109545250605615
Test R2-score is 0.9167827516081078+-0.003310002079504476
